## ReMECS -- Real-time Multimodal Emotion Classification Syatem

## Using multimodal data stream [EEG+EDA+RESP_BELT] 

## Using  ExtremelyFastDecisionTreeClassifier 

## Modifications:
- - New ensemble approach (Reward Penalty based Weighted Ensemble [RPWE])


## Date: 23 Sept 2021 at 10:10 AM

In [1]:
#============================
# Import important libraries
#============================
import pandas as pd 
import numpy as np
import math
import scipy
import pywt
from creme import metrics
import time
import datetime
from statistics import mode
from scipy import stats
from sklearn import preprocessing
from collections import defaultdict, Counter
from scipy.special import expit
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from window_slider import Slider

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values)> 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):    
    list_values = list_values[0,:]
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [3]:
#======================================================
# EDA Feature Extraction (Wavelet Features)
#======================================================
def extract_eda_features(raw_eda):
    features =[]
    EDA = raw_eda
    list_coeff = pywt.wavedec(EDA, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features


In [4]:
#======================================================
# RESP BELT Feature Extraction (Wavelet Features)
#======================================================

def extract_resp_belt_features(raw_data):
    features =[]
    resp_belt = raw_data
    list_coeff = pywt.wavedec(resp_belt, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features

In [5]:
def eeg_features(raw_data):
    ch = 0
    features= []
    def calculate_entropy(list_values):
        counter_values = Counter(list_values).most_common()
        probabilities = [elem[1]/len(list_values) for elem in counter_values]
        entropy=scipy.stats.entropy(probabilities)
        return entropy

    def calculate_statistics(list_values):
        median = np.nanpercentile(list_values, 50)
        mean = np.nanmean(list_values)
        std = np.nanstd(list_values)
        var = np.nanvar(list_values)
        rms = np.nanmean(np.sqrt(list_values**2))
        return [median, mean, std, var, rms]

    def get_features(list_values):    
    #     list_values = list_values[0,:]
        entropy = calculate_entropy(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + statistics
    
    for i in range(raw_data.shape[0]):
        ch_data = raw_data[i]
        list_coeff = pywt.wavedec(ch_data, 'db4', level=5)
        for coeff in list_coeff:
            features += get_features(coeff)
            
        ch = ch+1
    return features

In [6]:
# ##===================================================
# # EEG data read from files
# ##===================================================
# def eeg_data(p,v):
#     file_eeg = '/home/gp/Desktop/MER_arin/data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
#     print(file_eeg)
#     df = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
#     eeg_sig = df.loc[df.iloc[:,1] == v]
#     return eeg_sig

# ##===================================================
# # EDA data read from files
# ##===================================================
# def eda_data(p,v):
#     file_eda = '/home/gp/Desktop/MER_arin/data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
#     print(file_eda)
#     df = pd.read_csv(file_eda,sep=',', header = None, engine='python')
#     eda_sig = df.loc[df.iloc[:,1] == v]
#     return eda_sig

# ##===================================================
# # Resp data read from files
# ##===================================================
# def resp_data(p,v):
#     file_resp = '/home/gp/Desktop/MER_arin/data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
#     print(file_resp)
#     df = pd.read_csv(file_resp,sep=',', header = None, engine='python')
#     resp_sig = df.loc[df.iloc[:,1] == v]
#     return resp_sig

In [7]:
#================================================================
# Initialization of different parameters and performance metrics
#================================================================
segment_in_sec = 10 #in sec
bucket_size = int((8064/60)*segment_in_sec)  #8064 is for 60 sec record
overlap_count = 0

num_classifiers = 3 #Total number of classifiers
w_val =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers
w_aro =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers

cBbest_val = [] #Classifiers least error for valence classsification
cBest_aro = []  #Classifiers least error for arousal classsification
cWorst_val = [] #Classifiers heighest error for valence classsification
cWorst_aro = []  #Classifiers heighest error for arousal classsification

val_MSE = []
aro_MSE = []

beta_val = np.ones(num_classifiers)/num_classifiers
beta_aro = np.ones(num_classifiers)/num_classifiers

n_epoch =1
epochs =  1 #epoch is 1 because the model will be trained only once

optimizer= 'EFDT' #optimizer
classifier = 'EFDT_'+str(optimizer)
indx = 0
run = 1

participant = 32
videos = 40

global eeg_emotion, eda_emotion, resp_emotion, mer_emotion, all_eta
eeg_emotion = []
eda_emotion = []
resp_emotion = []
mer_emotion = []

all_eta =[]
init_m = 0

#================================================
# Performance matric declaration here
#================================================
mer_acc_val = metrics.Accuracy() #Accuracy
mer_f1m_val = metrics.F1() #F1 measure  
mer_acc_aro = metrics.Accuracy() #Accuracy
mer_f1m_aro = metrics.F1() #F1 measure
mer_roc_val = metrics.ROCAUC()
mer_roc_aro = metrics.ROCAUC() 
mer_mcc_val = metrics.MCC()
mer_mcc_aro = metrics.MCC()


eeg_acc_val = metrics.Accuracy() #Accuracy
eeg_f1m_val = metrics.F1() #F1 measure  
eeg_acc_aro = metrics.Accuracy() #Accuracy
eeg_f1m_aro = metrics.F1() #F1 measure
eeg_roc_val = metrics.ROCAUC()
eeg_roc_aro = metrics.ROCAUC()
eeg_mcc_val = metrics.MCC()
eeg_mcc_aro = metrics.MCC()

eda_acc_val = metrics.Accuracy() #Accuracy
eda_f1m_val = metrics.F1() #F1 measure  
eda_acc_aro = metrics.Accuracy() #Accuracy
eda_f1m_aro = metrics.F1() #F1 measure
eda_roc_val = metrics.ROCAUC()
eda_roc_aro = metrics.ROCAUC()
eda_mcc_val = metrics.MCC()
eda_mcc_aro = metrics.MCC()

resp_acc_val = metrics.Accuracy() #Accuracy
resp_f1m_val = metrics.F1() #F1 measure  
resp_acc_aro = metrics.Accuracy() #Accuracy
resp_f1m_aro = metrics.F1() #F1 measure
resp_roc_val = metrics.ROCAUC()
resp_roc_aro = metrics.ROCAUC()
resp_mcc_val = metrics.MCC()
resp_mcc_aro = metrics.MCC()

 
mer_cm_val  = metrics.ConfusionMatrix()
mer_cm_aro  = metrics.ConfusionMatrix()

eeg_cm_val  = metrics.ConfusionMatrix()
eeg_cm_aro  = metrics.ConfusionMatrix()

eda_cm_val  = metrics.ConfusionMatrix()
eda_cm_aro  = metrics.ConfusionMatrix()
  
resp_cm_val  = metrics.ConfusionMatrix()
resp_cm_aro  = metrics.ConfusionMatrix()
    
itr = 0 #controls the learning rate


for ii in range(0,participant):

        p =ii+1 #participant number
        
        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')
        ##===================================================
        # EEG data read from files
        ##===================================================
        def eeg_data(p):
            file_eeg = '/home/gp/Desktop/MER_arin/DEAP_data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
            print(file_eeg)
            eeg_sig = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
            return eeg_sig

        ##===================================================
        # EDA data read from files
        ##===================================================
        def eda_data(p):
            file_eda = '/home/gp/Desktop/MER_arin/DEAP_data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
            print(file_eda)
            eda_sig = pd.read_csv(file_eda,sep=',', header = None, engine='python')
            return eda_sig

        ##===================================================
        # Resp data read from files
        ##===================================================
        def resp_data(p):
            file_resp = '/home/gp/Desktop/MER_arin/DEAP_data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
            print(file_resp)
            resp_sig = pd.read_csv(file_resp,sep=',', header = None, engine='python')
            return resp_sig
        
        #------------------------------------
        # Once file fetched data stored here 
        #------------------------------------
        grand_eeg = eeg_data(p)
        grand_eda = eda_data(p)
        grand_resp = resp_data(p)

        
        for jj in range(0,videos):
            v = jj+1 #Video number
            print('=========================================================================')
            p_v = 'Person:'+ ' ' +str(p)+ ' ' +'Video:'+str(v)
            print(p_v)
            
            emotion_label =[]
            
            
            ##===================================================
            # Data read from files
            ##===================================================
            eeg_sig = grand_eeg.loc[grand_eeg.iloc[:,1] == v]
            eda_sig = grand_eda.loc[grand_eda.iloc[:,1] == v]
            resp_sig = grand_resp.loc[grand_resp.iloc[:,1] == v]
            
            #=================================================
            #emotion labels (valence, arousal) mapping 0-1
            #=================================================
            val = eeg_sig.iloc[0,8067]
            aro = eeg_sig.iloc[0,8068]
            
            #valence emotion maping 0-> low valence and 1-> high valence

            if (val >5):
                vl = 1 #high valence
            else:
                vl = 0 #low valence

            #arousal emotion maping 0-> low arousal and 1-> high high arousal
            if (aro >5):
                al = 1 #high arousal
            else:
                al = 0 #low arousal
                
            y_act_val = np.array([vl])
            y_act_aro = np.array([al])
            
            #==========================================================
            # Predicted Valence and Arousal labels list initialization
            #==========================================================
            eeg_val_prdt=[]
            eeg_aro_prdt =[]
            
            eda_val_prdt=[]
            eda_aro_prdt =[]
            
            resp_val_prdt=[]
            resp_aro_prdt =[]
            
            
            #=========================================
            # Sliding window starts here 
            #=========================================
            slider_eeg = Slider(bucket_size,overlap_count)
            slider_eda = Slider(bucket_size,overlap_count)
            slider_resp = Slider(bucket_size,overlap_count)
            
            eeg_sig = np.array(eeg_sig.iloc[range(0,32),range(3,8067)]) #keeping only eeg signals
            eda_sig = np.array(eda_sig.iloc[:,range(3,8067)]) #keeping only eda signals
            resp_sig = np.array(resp_sig.iloc[:,range(3,8067)]) #keeping only resp signals
            
            slider_eeg.fit(eeg_sig)
            slider_eda.fit(eda_sig)
            slider_resp.fit(resp_sig)

            while True:
                window_data_eeg = slider_eeg.slide()
                window_data_eda = slider_eda.slide() 
                window_data_resp = slider_resp.slide() 
                
                #=================================================
                # Feature extraction from EEG
                #=================================================
                features_eeg = eeg_features(window_data_eeg)
                x_eeg = np.array([features_eeg])  #EEG raw feature vector
#                 x_eeg = preprocessing.normalize(eeg) # EEG normalized features [0,1] 
                
                
                #=================================================
                # Feature extraction from EDA
                #=================================================
                eda_features = extract_eda_features(np.array(window_data_eda))
                x_eda = np.array([eda_features]) #EDA raw feature vector
#                 x_eda = preprocessing.normalize(eda) #EDA normalized features
                
                #=================================================
                # Feature extraction from Resp belt
                #=================================================

                resp_features = extract_resp_belt_features(np.array(window_data_resp))
                x_resp = np.array([resp_features]) #RESP BELT raw feature vector
#                 x_resp = preprocessing.normalize(resp) #RESP BELT normalized features
            
            
                #===================================================
                # Model initialization
                #===================================================
                if init_m == 0:
                    print('EEG Feature shape{}:'.format(x_eeg.shape))
                    print('EDA Feature shape{}:'.format(x_eda.shape))
                    print('RESP BELT Feature shape{}:'.format(x_resp.shape))

                    #========================
                    # For EEG data MLP model
                    #========================
                    eeg_model_val = ExtremelyFastDecisionTreeClassifier()
                    eeg_model_aro = ExtremelyFastDecisionTreeClassifier()

                    #========================
                    # For EDA data MLp model
                    #========================
                    eda_model_val = ExtremelyFastDecisionTreeClassifier()
                    eda_model_aro = ExtremelyFastDecisionTreeClassifier()
                    

                    #==============================
                    # For Resp Belt data MLP Model
                    #==============================
                    resp_model_val = ExtremelyFastDecisionTreeClassifier()
                    resp_model_aro = ExtremelyFastDecisionTreeClassifier()


                    init_m = init_m+1


                #===============================================================
                # Emotion Classification --> Valence and Arousal
                #===============================================================


                #===========================================
                # From EEG data -- RECS System
                #===========================================

                #Valence classification EEG

                #Test the model first 
                if (indx == 0): #When the model sees the first data
                    tmp_eeg_val = 0
                    y_pred_val_eeg = 2 #other than what are class labels
                else: 
                    eeg_val = eeg_model_val.predict_proba(x_eeg)
                    tmp_eeg_val = np.max(eeg_val)
                    y_pred_val_eeg = np.argmax(eeg_val)
                    
                #Train the model once
                eeg_model_val.partial_fit(x_eeg,y_act_val)

                eeg_acc_val = eeg_acc_val.update(y_act_val[0], y_pred_val_eeg)  # update the accuracy metric

                eeg_f1m_val = eeg_f1m_val.update(y_act_val[0], y_pred_val_eeg) #update f1 measure metric
                
                eeg_roc_val = eeg_roc_val.update(y_act_val[0], y_pred_val_eeg)
                
                eeg_mcc_val = eeg_mcc_val.update(y_act_val[0], y_pred_val_eeg)

                #Arousal classification EEG

                if (indx == 0): #When the model sees the first data
                    tmp_eeg_aro = 0
                    y_pred_aro_eeg = 2 #other than what are class labels
                else: 
                    eeg_aro = eeg_model_aro.predict_proba(x_eeg)
                    tmp_eeg_aro = np.max(eeg_aro)
                    y_pred_aro_eeg = np.argmax( eeg_aro)

                #Train the model once
                eeg_model_aro.partial_fit(x_eeg,y_act_aro)

                eeg_acc_aro = eeg_acc_aro.update(y_act_aro[0], y_pred_aro_eeg)  # update the accuracy metric

                eeg_f1m_aro = eeg_f1m_aro.update(y_act_aro[0], y_pred_aro_eeg) #update f1 measure metric
                
                eeg_roc_aro = eeg_roc_aro.update(y_act_aro[0], y_pred_aro_eeg)
                
                eeg_mcc_aro = eeg_mcc_aro.update(y_act_aro[0], y_pred_aro_eeg)

                #===========================================
                # From EDA data 
                #===========================================

                #Valence classification EDA

                if (indx == 0): #When the model sees the first data
                    tmp_eda_val = 0
                    y_pred_val_eda = 2 #other than what are class labels
                else: 
                    eda_val = eda_model_val.predict_proba(x_eda) 
                    tmp_eda_val = np.max(eda_val)
                    y_pred_val_eda = np.argmax( eda_val)

                #Train the model once
                eda_model_val.partial_fit(x_eda,y_act_val)

                eda_acc_val = eda_acc_val.update(y_act_val[0], y_pred_val_eda)  # update the accuracy metric

                eda_f1m_val = eda_f1m_val.update(y_act_val[0], y_pred_val_eda) #update f1 measure metric
                
                eda_roc_val = eda_roc_val.update(y_act_val[0], y_pred_val_eda)
                
                eda_mcc_val = eda_mcc_val.update(y_act_val[0], y_pred_val_eda)
                

                #Arousal classification EDA
                if (indx == 0): #When the model sees the first data
                    tmp_eda_aro = 0
                    y_pred_aro_eda = 2 #other than what are class labels
                else: 
                    eda_aro = eda_model_aro.predict_proba(x_eda)
                    tmp_eda_aro = np.max(eda_aro)
                    y_pred_aro_eda = np.argmax(eda_aro)

                #Train the model once
                eda_model_aro.partial_fit(x_eda,y_act_aro)

                eda_acc_aro = eda_acc_aro.update(y_act_aro[0], y_pred_aro_eda)  # update the accuracy metric

                eda_f1m_aro = eda_f1m_aro.update(y_act_aro[0], y_pred_aro_eda) #update f1 measure metric
                
                eda_roc_aro = eda_roc_aro.update(y_act_aro[0], y_pred_aro_eda)
                
                eda_mcc_aro = eda_mcc_aro.update(y_act_aro[0], y_pred_aro_eda)

                #===========================================
                # From Resp Belt data
                #===========================================

                #Valence classification Resp Belt

                if (indx == 0): #When the model sees the first data
                    tmp_resp_val = 0
                    y_pred_val_resp = 2 #other than what are class labels
                else: 
                    resp_val  = resp_model_val.predict_proba(x_resp)
                    tmp_resp_val = np.max(resp_val)
                    y_pred_val_resp = np.argmax(resp_val)

                #Train the model once
                resp_model_val.partial_fit(x_resp,y_act_val)

                resp_acc_val = resp_acc_val.update(y_act_val[0], y_pred_val_resp)  # update the accuracy metric

                resp_f1m_val = resp_f1m_val.update(y_act_val[0], y_pred_val_resp) #update f1 measure metric
                
                resp_roc_val = resp_roc_val.update(y_act_val[0], y_pred_val_resp)
                
                resp_mcc_val = resp_roc_val.update(y_act_val[0], y_pred_val_resp)
                

                #Arousal classification Resp Belt
                if (indx == 0): #When the model sees the first data
                    tmp_resp_aro = 0
                    y_pred_aro_resp = 2 #other than what are class labels
                else: 
                    resp_aro = resp_model_aro.predict_proba(x_resp)
                    tmp_resp_aro = np.max(resp_aro)
                    y_pred_aro_resp = np.argmax(resp_aro)
               

                #Train the model once
                resp_model_aro.partial_fit(x_resp,y_act_aro)

                resp_acc_aro = resp_acc_aro.update(y_act_aro[0], y_pred_aro_resp)  # update the accuracy metric

                resp_f1m_aro = resp_f1m_aro.update(y_act_aro[0], y_pred_aro_resp) #update f1 measure metric
                
                resp_roc_aro = resp_roc_aro.update(y_act_aro[0], y_pred_aro_resp)
                
                resp_mcc_aro = resp_mcc_aro.update(y_act_aro[0], y_pred_aro_resp)
                
                
                indx = indx+1

                if slider_eeg.reached_end_of_list(): break
                
            #===================================================
            #   Error storing and comparing Part of each models
            #===================================================
            
            eeg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eeg_val))
            eeg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eeg_aro))
            
            eda_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eda_val))
            eda_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eda_aro))
            
            resp_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_resp_val))
            resp_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_resp_aro))
            
            val_MSE = [eeg_val_MSE, eda_val_MSE, resp_val_MSE] #Valence errors 
            aro_MSE = [eeg_aro_MSE, eda_aro_MSE, resp_aro_MSE] #Arousal errors
            

            
            #========================================================
            # Valence Arousal MSE Compariosn and storing in Cbest and Cworst
            #========================================================
            
            if (jj==0): #First Video
                cBest_val = val_MSE
                cBest_aro = aro_MSE
                
                cWorst_val  = val_MSE
                cWorst_aro = aro_MSE
                
            else:
                #-----------------------------------------
                #Beta for calence and arousal calculation
                #-----------------------------------------              
                beta_val = np.true_divide(list(np.subtract(cBest_val,val_MSE)), (1+np.exp(list(np.subtract(cWorst_val,cBest_val)))))
                beta_aro = np.true_divide(list(np.subtract(cBest_aro,aro_MSE)), (1+np.exp(list(np.subtract(cWorst_aro,cBest_aro)))))
                
                #--------------------------------------------------------------
                # Classifiers best MSE error and worst MSE error value update
                #-------------------------------------------------------------
                cBest_val = np.minimum(cBest_val, val_MSE)
                cBest_aro = np.minimum(cBest_aro, aro_MSE)
                cWorst_val = np.maximum(cWorst_val, val_MSE)
                cWorst_aro = np.maximum(cWorst_aro, aro_MSE)
            ##=============================
            # Confusion Matric Calculation
            ##=============================
            
            eeg_cm_val = eeg_cm_val.update(y_act_val[0], y_pred_val_eeg)
            eeg_cm_aro = eeg_cm_aro.update(y_act_aro[0], y_pred_aro_eeg)
            
            eda_cm_val = eda_cm_val.update(y_act_val[0], y_pred_val_eda)
            eda_cm_aro = eda_cm_aro.update(y_act_aro[0], y_pred_aro_eda)
            
            resp_cm_val = resp_cm_val.update(y_act_val[0], y_pred_val_resp)
            resp_cm_varo = resp_cm_aro.update(y_act_aro[0], y_pred_aro_resp)            
            
            #=============================================================
            # Storing All Results
            #=============================================================
            
            emotion_label.append([y_pred_val_eeg,y_pred_aro_eeg]) #appending valence & arousal predicted EEG
            emotion_label.append([y_pred_val_eda, y_pred_aro_eda]) #appending valence & arousal predicted EDA
            emotion_label.append([y_pred_val_resp, y_pred_aro_resp]) #appending valence & arousal predicted resp

            emotion_label = np.array(emotion_label)
            
                    
            #====================================================================
            # Decision label ensemble --> Reward Penalty Based Weightrd Ensemble
            #====================================================================
            val_label = emotion_label[:,0]
            aro_label = emotion_label[:,1]
            
            #------------------------------------------
            # Valence Class ensemble
            #------------------------------------------
            
            p_val = np.dot([w_val],val_label)
            y_prdt_mer_val = (p_val -0.5)
            if(y_prdt_mer_val > 0):
                mer_val = 1
            else:
                mer_val = 0

            #============================================
            # Weight update for Valence
            #============================================
            w_val = w_val + list(np.multiply(w_val,beta_val))    
            w_val_sum = sum(w_val) #total sum of weights
            w_val = np.array(w_val/w_val_sum) #weight rescaling


            
            #------------------------------------------
            # Arousal Class ensemble
            #------------------------------------------            
            p_val = np.dot([w_aro],aro_label)
            y_prdt_mer_aro = (p_val-0.5)
            if(y_prdt_mer_aro > 0):
                mer_aro = 1
            else:
                mer_aro = 0

            w_aro = w_aro + list(np.multiply(w_aro,beta_aro))
            w_aro_sum = sum(w_aro) #total sum of weights
            w_aro = np.array(w_aro/w_aro_sum) #weight rescaling
            
            #========================================================
            # ReMECS performance metric 
            #========================================================
            mer_acc_val = mer_acc_val.update(y_act_val[0], mer_val)
            mer_f1m_val = mer_f1m_val.update(y_act_val[0], mer_val)
            mer_roc_val = mer_roc_val.update(y_act_val[0], mer_val)
            mer_mcc_val = mer_mcc_val.update(y_act_val[0], mer_val)
            mer_cm_val = mer_cm_val.update(y_act_val[0], mer_val)
            
            mer_acc_aro = mer_acc_aro.update(y_act_aro[0], mer_aro)
            mer_f1m_aro = mer_f1m_aro.update(y_act_aro[0], mer_aro)
            mer_roc_aro = mer_roc_aro.update(y_act_aro[0], mer_aro)
            mer_mcc_aro = mer_mcc_aro.update(y_act_aro[0], mer_aro)
            mer_cm_aro = mer_cm_aro.update(y_act_aro[0], mer_aro)
            
            eeg_emotion.append(np.array([p,v,eeg_acc_val.get(), eeg_f1m_val.get(), eeg_roc_val.get(),eeg_mcc_val.get(),
                                         eeg_acc_aro.get(), eeg_f1m_aro.get(),eeg_roc_aro.get(), eeg_mcc_aro.get(), 
                                         y_act_val[0], y_pred_val_eeg, y_act_aro[0], y_pred_aro_eeg]))
            
            eda_emotion.append(np.array([p,v,eda_acc_val.get(), eda_f1m_val.get(), eda_roc_val.get(),eda_mcc_val.get(),
                                         eda_acc_aro.get(), eda_f1m_aro.get(), eda_roc_aro.get(), eda_mcc_aro.get(),
                                         y_act_val[0], y_pred_val_eda, y_act_aro[0], y_pred_aro_eda]))
            
            resp_emotion.append(np.array([p,v, resp_acc_val.get(), resp_f1m_val.get(), resp_roc_val.get(), resp_mcc_val.get(),
                                          resp_acc_aro.get(),resp_f1m_aro.get(), resp_roc_aro.get(), resp_mcc_aro.get(), 
                                          y_act_val[0], y_pred_val_resp, y_act_aro[0], y_pred_aro_resp]))
            
            mer_emotion.append(np.array([p,v,mer_acc_val.get(), mer_f1m_val.get(), mer_roc_val.get(), mer_mcc_val.get(),
                                         mer_acc_aro.get(), mer_f1m_aro.get(), mer_roc_aro.get(), mer_mcc_aro.get(), 
                                         y_act_val[0], mer_val, y_act_aro[0], mer_aro]))
            print('Actual emotion [valence: {}, arousal: {}]'.format(y_act_val, y_act_aro))
            print('Single modals Predicted emotion')
            print('[valence, arousal]:')
            print(emotion_label)
            
            print('MER Predicted emotion [valence: {}, arousal: {}]'.format(mer_val, mer_aro))
            print('Val MSE',val_MSE)
            print('Aro MSE',aro_MSE)
            # print('------------------------------')
            print('Beta_Val:', beta_val)
            print('Beta_Aro:', beta_aro)
            print('------------------------------')
            print('W Val:', w_val)
            print('W Aro:', w_aro)
            # print('------------------------------')
            # print('CBest Val MSE',cBest_val)
            # print('CBest Aro MSE',cBest_aro)
            # print('------------------------------')
            # print('Cworst Val MSE',cWorst_val)
            # print('Cworst Aro MSE',cWorst_aro)
            # print('==============================================')            
            # print('..................................................')
            # print('EEG')

            # print("Valence Accuracy:{}".format(round(eeg_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eeg_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eeg_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eeg_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eeg_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eeg_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eeg_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eeg_mcc_aro.get(),4)))

            # print('..................................................')
            # print('EDA')

            # print("Valence Accuracy:{}".format(round(eda_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eda_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eda_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eda_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eda_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eda_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eda_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eda_mcc_aro.get(),4)))


            # print('..................................................')
            # print('RESP BELT')

            # print("Valence Accuracy:{}".format(round(resp_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(resp_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(resp_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(resp_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(resp_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(resp_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(resp_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(resp_mcc_aro.get(),4)))

            # print('..................................................')
            print('MER model')

            print("Valence Accuracy:{}".format(round(mer_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(mer_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(mer_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(mer_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(mer_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(mer_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(mer_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(mer_mcc_aro.get(),4)))
            print('===============================xxxx================================')
                
                
                
            



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/MER_arin/DEAP_data/eeg_data/1_data_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/eda_data/1_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/1_Respiration_data_from_DEAP.csv
Person: 1 Video:1
EEG Feature shape(1, 1152):
EDA Feature shape(1, 48):
RESP BELT Feature shape(1, 48):
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.5, 0.0, 0.0]
Aro MSE [0.5, 0.0, 0.0]
Beta_Val: [0.33333333 0.33333333 0.33333333]
Beta_Aro: [0.33333333 0.33333333 0.33333333]
------------------------------
W Val: [0.33333333 0.33333333 0.33333333]
W Aro: [0.33333333 0.33333333 0.33333333]
MER model
Valence Accuracy:1.0
Valence F1 score:1.0
Valence ROC score:-0.0
Valence MCC score:0.0
Arousal Accuracy:1.0
Arousal F1 score:1.0
Arousal ROC score:-0.0
Ar

Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [1 1]
 [0 1]]
MER Predicted emotion [valence: 1, arousal: 0]
Val MSE [0.20875739644970417, 0.4999999999999656, 0.4999898036627798]
Aro MSE [0.0, 0.5, 0.43060047577298266]
Beta_Val: [-0.07881441 -0.18877033 -0.18876648]
Beta_Aro: [ 0.         -0.18877033 -0.16256919]
------------------------------
W Val: [0.45538763 0.24800126 0.29661111]
W Aro: [0.71658598 0.12254132 0.1608727 ]
MER model
Valence Accuracy:0.7273
Valence F1 score:0.8235
Valence ROC score:0.625
Valence MCC score:0.4183
Arousal Accuracy:1.0
Arousal F1 score:1.0
Arousal ROC score:1.0
Arousal MCC score:1.0
===============================xxxx================================
Person: 1 Video:12
Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.0, 0.49999998360610876, 0.4851779349743701]
Aro MSE [0.0, 

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 0]
Val MSE [0.5, 1.6458895144984576e-06, 9.749843026130228e-19]
Aro MSE [0.5, 0.006582851275762955, 1.3868544553964468e-20]
Beta_Val: [-1.88770334e-01 -6.21390228e-07 -3.68096226e-19]
Beta_Aro: [-1.88770334e-01 -2.48529407e-03 -5.23593959e-21]
------------------------------
W Val: [0.44580393 0.24278189 0.31141419]
W Aro: [0.47780418 0.18819506 0.33400076]
MER model
Valence Accuracy:0.6667
Valence F1 score:0.6957
Valence ROC score:0.6667
Valence MCC score:0.3303
Arousal Accuracy:0.6667
Arousal F1 score:0.5333
Arousal ROC score:0.6818
Arousal MCC score:0.4625
===============================xxxx================================
Person: 1 Video:22
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.0849600839

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.17040175310445582, 0.5, 0.499995267151829]
Aro MSE [0.5, 2.399714873682318e-27, 0.020054367295347753]
Beta_Val: [-0.06433359 -0.18877033 -0.18876855]
Beta_Aro: [-1.88770334e-01 -9.05989958e-28 -7.57133924e-03]
------------------------------
W Val: [0.51306085 0.19087378 0.29606536]
W Aro: [0.47306814 0.18631624 0.34061562]
MER model
Valence Accuracy:0.6774
Valence F1 score:0.7222
Valence ROC score:0.6688
Valence MCC score:0.3376
Arousal Accuracy:0.6452
Arousal F1 score:0.4762
Arousal ROC score:0.6562
Arousal MCC score:0.4246
===============================xxxx================================
Person: 1 Video:32
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 0]
 [0 0]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 0]
Val MSE [0.1598640388147255, 0.5, 0.38438

/home/gp/Desktop/MER_arin/DEAP_data/eda_data/2_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/2_Respiration_data_from_DEAP.csv
Person: 2 Video:1
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [1 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.5, 0.0, 4.039038861728307e-06]
Aro MSE [0.0767680133277801, 0.0, 0.0767680133277801]
Beta_Val: [-1.88770334e-01  0.00000000e+00 -2.39347892e-10]
Beta_Aro: [-0.03045653  0.         -0.01156831]
------------------------------
W Val: [0.75619236 0.09046385 0.15334379]
W Aro: [0.31033716 0.21905391 0.47060893]
MER model
Valence Accuracy:0.6585
Valence F1 score:0.65
Valence ROC score:0.6583
Valence MCC score:0.3167
Arousal Accuracy:0.6585
Arousal F1 score:0.6111
Arousal ROC score:0.72
Arousal MCC score:0.4844
===============================xxxx================================
Person: 2 Video:2
Actual emotion [valence: [1], arousal: [1]]
Single modals Pre

...

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.0, 0.49516585804351726, 0.5]
Aro MSE [0.5, 2.4448853492738046e-10, 0.0]
Beta_Val: [ 0.         -0.18694525 -0.18877033]
Beta_Aro: [-1.88770334e-01 -9.23043650e-11  0.00000000e+00]
------------------------------
W Val: [1.00000000e+00 1.55097546e-22 2.97384885e-10]
W Aro: [9.99999630e-01 2.19082234e-08 3.48009034e-07]
MER model
Valence Accuracy:0.6504
Valence F1 score:0.698
Valence ROC score:0.6408
Valence MCC score:0.2854
Arousal Accuracy:0.6164
Arousal F1 score:0.5919
Arousal ROC score:0.6374
Arousal MCC score:0.2817
===============================xxxx================================
Person: 31 Video:34
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [1 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.0, 0.493283811621123, 0.5]
Aro MSE [

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [1 1]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.5, 0.04170891377909798, 0.0]
Aro MSE [0.5, 8.94827559452176e-20, 0.10081291173525901]
Beta_Val: [-0.18877033 -0.00181353  0.        ]
Beta_Aro: [-0.19567885  0.25        0.02540398]
------------------------------
W Val: [1.00000000e+00 3.94904165e-23 4.12550307e-11]
W Aro: [9.99997271e-01 2.27790688e-07 2.50095437e-06]
MER model
Valence Accuracy:0.6516
Valence F1 score:0.697
Valence ROC score:0.643
Valence MCC score:0.2894
Arousal Accuracy:0.613
Arousal F1 score:0.5878
Arousal ROC score:0.635
Arousal MCC score:0.2772
===============================xxxx================================
Person: 32 Video:4
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.5, 2.4980627377681667e-05, 0.0]
Aro MS

Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [1 0]
 [1 1]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.5, 4.153155211995758e-06, 7.06385371372271e-21]
Aro MSE [0.0, 0.4999400282264314, 0.1515970616580834]
Beta_Val: [-1.88770334e-01 -1.53295240e-06 -2.66689206e-21]
Beta_Aro: [ 0.         -0.18874769 -0.02493335]
------------------------------
W Val: [1.00000000e+00 1.72572242e-22 1.44748414e-10]
W Aro: [9.99994529e-01 1.91842755e-07 5.27944355e-06]
MER model
Valence Accuracy:0.648
Valence F1 score:0.6931
Valence ROC score:0.6396
Valence MCC score:0.282
Arousal Accuracy:0.6129
Arousal F1 score:0.5865
Arousal ROC score:0.635
Arousal MCC score:0.2776
===============================xxxx================================
Person: 32 Video:14
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [1 0]
 [1 1]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.5, 2.744

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [1 0]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.0, 0.15930698318556188, 0.5]
Aro MSE [0.5, 0.055593705026579794, 0.09354763921496917]
Beta_Val: [ 0.         -0.06014542 -0.18877033]
Beta_Aro: [-0.18877033 -0.02098888 -0.03531804]
------------------------------
W Val: [1.00000000e+00 1.41264235e-22 1.03640284e-10]
W Aro: [9.99981470e-01 5.15933939e-07 1.80145573e-05]
MER model
Valence Accuracy:0.6485
Valence F1 score:0.6938
Valence ROC score:0.6399
Valence MCC score:0.2826
Arousal Accuracy:0.6105
Arousal F1 score:0.5838
Arousal ROC score:0.6334
Arousal MCC score:0.2746
===============================xxxx================================
Person: 32 Video:24
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [1 0]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.0, 0.2239541913701258, 0.5]
Aro M

Actual emotion [valence: [0], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [1 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.0, 0.4666432748358446, 0.4999999995865628]
Aro MSE [0.0, 0.17126538233061686, 0.183780126418651]
Beta_Val: [ 0.         -0.17617843 -0.18877033]
Beta_Aro: [ 0.         -0.06465965 -0.06938447]
------------------------------
W Val: [1.00000000e+00 7.65088737e-23 4.68572317e-11]
W Aro: [9.99958437e-01 1.24292415e-06 4.03205706e-05]
MER model
Valence Accuracy:0.6496
Valence F1 score:0.6933
Valence ROC score:0.6418
Valence MCC score:0.2861
Arousal Accuracy:0.608
Arousal F1 score:0.5803
Arousal ROC score:0.6317
Arousal MCC score:0.2714
===============================xxxx================================
Person: 32 Video:34
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.0, 0.19825016055771175,

In [8]:
print('EEG Val CM:')
print(eeg_cm_val)

print('EEG aro CM:')
print(eeg_cm_aro)

print('EDA Val CM:')
print(eda_cm_val)

print('EDA aro CM:')
print(eda_cm_aro)

print('RESP Val CM:')
print(resp_cm_val)
print('RESP Aro CM:')
print(resp_cm_aro)

print('ReMECS Val CM:')
print(mer_cm_val)

print('ReMECS aro CM:')
print(mer_cm_aro)

EEG Val CM:
             0      1
      0    428    144
      1    283    425
EEG aro CM:
             0      1
      0    425    118
      1    396    341
EDA Val CM:
             0      1
      0    339    233
      1    132    576
EDA aro CM:
             0      1
      0    365    178
      1    153    584
RESP Val CM:
             0      1
      0    259    313
      1     85    623
RESP Aro CM:
             0      1
      0    349    194
      1    235    502
ReMECS Val CM:
             0      1
      0    329    243
      1    204    504
ReMECS aro CM:
             0      1
      0    430    113
      1    391    346


In [9]:
print('[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]')
#=============================
# EEG
#=============================
t_eeg_emotion =pd.DataFrame(eeg_emotion) 
t_eeg_emotion = t_eeg_emotion.iloc[:,range(2,10)]
t_eeg_emotion_avg = np.array(np.mean(t_eeg_emotion))
t_eeg_emotion_std = np.array(np.std(t_eeg_emotion))

print('EEG result Average:', t_eeg_emotion_avg)
print('EEG result std:', t_eeg_emotion_std)

#=============================
# EDA
#=============================
t_eda_emotion =pd.DataFrame(eda_emotion) 
t_eda_emotion = t_eda_emotion.iloc[:,range(2,10)]
t_eda_emotion_avg = np.array(np.mean(t_eda_emotion))
t_eda_emotion_std = np.array(np.std(t_eda_emotion))

print('EDA result Average:', t_eda_emotion_avg)
print('EDA result std:', t_eda_emotion_std)
#=============================
# RESP BELT
#=============================
t_resp_emotion =pd.DataFrame(resp_emotion) 
t_resp_emotion = t_resp_emotion.iloc[:,range(2,10)]
t_resp_emotion_avg = np.array(np.mean(t_resp_emotion))
t_resp_emotion_std = np.array(np.std(t_resp_emotion))

print('Resp Belt result Average:', t_resp_emotion_avg)
print('Resp Belt result std:', t_resp_emotion_std)
#=============================
# MER
#=============================
t_mer_emotion =pd.DataFrame(mer_emotion) 
t_mer_emotion = t_mer_emotion.iloc[:,range(2,10)]
t_mer_emotion_avg = np.array(np.mean(t_mer_emotion))
t_mer_emotion_std = np.array(np.std(t_mer_emotion))

print('MER result Average:', t_mer_emotion_avg)
print('MER result std:', t_mer_emotion_std)

[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]
EEG result Average: [0.60528248 0.60528988 0.61229674 0.22571179 0.54901013 0.47020905
 0.56175029 0.1316287 ]
EEG result std: [0.04606123 0.05331919 0.0421266  0.06292249 0.04071852 0.10606827
 0.04334312 0.06222948]
EDA result Average: [0.65469547 0.70100777 0.64474836 0.29676019 0.64678539 0.65387791
 0.63996948 0.28629159]
EDA result std: [0.02669196 0.02142122 0.04201565 0.05659581 0.02433548 0.06279889
 0.03974422 0.03925567]
Resp Belt result Average: [0.58334777 0.6598449  0.56419455 0.56419455 0.57495631 0.55587599
 0.57544937 0.15411678]
Resp Belt result std: [0.03302152 0.04974112 0.04087846 0.04087846 0.02892476 0.0835878
 0.04117301 0.04849615]
MER result Average: [0.63839707 0.68199036 0.62877192 0.2650092  0.60710376 0.55119612
 0.61615225 0.24694995]
MER result std: [0.04253683 0.03490893 0.05180506 0.08406472 0.04931707 0.07930258
 0.05730109 0.0931082 ]


#  Result Saving all

In [10]:
time_stamp = datetime.datetime.now()
run =1
algo= 'EFDT/' 
filename = "/home/gp/Desktop/MER_arin/mer_codes/12DEAP_RPWE/Results_RPWE_DEAP/"+algo
fname_eeg = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EEG'+'_' +classifier+'_results.csv'
fname_eda = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EDA'+'_' +classifier+'_results.csv'
fname_resp = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_RESP_BELT'+'_' +classifier+'_results.csv'
fname_mer = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_MER'+'_' +classifier+'_results.csv'

column_names = ['Person', 'Video', 'Acc_val', 'F1_val','roc_val', 'mcc_val','Acc_aro',
                'F1_aro', 'roc_aro', 'mcc_aro', 'y_act_val', 'y_pred_val', 'y_act_aro', 'y_pred_aro']

eeg_emotion = pd.DataFrame(eeg_emotion,columns = column_names)
eda_emotion = pd.DataFrame(eda_emotion,columns = column_names)
resp_emotion = pd.DataFrame(resp_emotion,columns = column_names)
mer_emotion = pd.DataFrame(mer_emotion,columns = column_names)

eeg_emotion.to_csv(fname_eeg)
eda_emotion.to_csv(fname_eda)
resp_emotion.to_csv(fname_resp)
mer_emotion.to_csv(fname_mer)


